In [ ]:
import sys
import os

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import seaborn as sns

sys.path.insert(0, os.path.abspath('..'))  # Add the absolute path of the parent directory to sys.path
from modules import elo

# Load Df

In [ ]:
# load games log
excel_file = r'/2023_10_02-fb_games.xlsx'
game_log = pd.read_excel('../data' + excel_file)


# load leaderboard
# Roster can be auto generated from game_log
leaderboard = pd.read_excel('../data/Roster.xlsx')  # auto gen this file

# Elo calculation

In [ ]:
# Initialise elo history
elo_history = pd.DataFrame(columns=['date', 'player', 'elo'])
elo_history.player = leaderboard.player
elo_history.elo = 1500
elo_history.date = '2023-10-01'

In [ ]:
# Initial Ratings
ratings = leaderboard.copy()
ratings = ratings.set_index('player')
ratings['elo'] = ratings['elo'].astype(float)
games = {}

for index, row in game_log.iterrows():
    # Get player names, ratings, and scores
    player1 = row["Player1"]
    player2 = row["Player2"]
    team1_score = row["Score1"]
    
    player3 = row["Player3"]
    player4 = row["Player4"]
    team2_score = row["Score2"]

    player1_rating = ratings.loc[player1, "elo"]
    player2_rating = ratings.loc[player2, "elo"]
    player3_rating = ratings.loc[player3, "elo"]
    player4_rating = ratings.loc[player4, "elo"]
    
    # Calculate the expected scores for the players
    player1_expected_score = elo.mean_expected_score(
        ratings.loc[player1, "elo"],
        ratings.loc[player3, "elo"],
        ratings.loc[player4, "elo"],
    )
    player2_expected_score = elo.mean_expected_score(
        ratings.loc[player2, "elo"],
        ratings.loc[player3, "elo"],
        ratings.loc[player4, "elo"],
    )
    player3_expected_score = elo.mean_expected_score(
        ratings.loc[player3, "elo"],
        ratings.loc[player1, "elo"],
        ratings.loc[player2, "elo"],
    )
    player4_expected_score = elo.mean_expected_score(
        ratings.loc[player4, "elo"],
        ratings.loc[player1, "elo"],
        ratings.loc[player2, "elo"],
    )
    
    # Calculate the team expected scores
    team1_expected_score = np.mean([player1_expected_score,
                                    player2_expected_score])
    team2_expected_score = np.mean([player3_expected_score,
                                    player4_expected_score])
    
    
    # Calculate the point factor to be used as a variable
    score_difference = abs(team1_score - team2_score)
    point_factor = elo.calculate_point_factor(score_difference)
    
    # Calculate the K value for each player based on the number of games played and their rating
    k1 = 50 / (1 + len(elo.specific_player_games(player1, index, game_log)) / 100)
    k2 = 50 / (1 + len(elo.specific_player_games(player2, index, game_log)) / 100) 
    k3 = 50 / (1 + len(elo.specific_player_games(player3, index, game_log)) / 100) 
    k4 = 50 / (1 + len(elo.specific_player_games(player4, index, game_log)) / 100)
    
    # Logg the wining team
    if team1_score > team2_score:
        team1_actual_score = 1
        team2_actual_score = 0
        ratings.loc[player1, 'wins'] += 1
        ratings.loc[player2, 'wins'] += 1
        
    else:
        team1_actual_score = 0
        team2_actual_score = 1
        ratings.loc[player3, 'wins'] += 1
        ratings.loc[player4, 'wins'] += 1
    
    # Logg total games
    ratings.loc[player1, 'total_games'] += 1
    ratings.loc[player2, 'total_games'] += 1
    ratings.loc[player3, 'total_games'] += 1
    ratings.loc[player4, 'total_games'] += 1
    
    # Calculate the new Elo ratings for each player
    player1_new_rating = player1_rating + k1 * point_factor  * (team1_actual_score - team1_expected_score)
    player2_new_rating = player2_rating + k2 * point_factor  * (team1_actual_score - team1_expected_score)
    player3_new_rating = player3_rating + k3 * point_factor  * (team2_actual_score - team2_expected_score)
    player4_new_rating = player4_rating + k4 * point_factor  * (team2_actual_score - team2_expected_score)
    
    ratings.loc[player1, 'elo'] = player1_new_rating
    ratings.loc[player2, 'elo'] = player2_new_rating
    ratings.loc[player3, 'elo'] = player3_new_rating
    ratings.loc[player4, 'elo'] = player4_new_rating
    
    # Update elo change column
    ratings.loc[player1, 'change'] = int(player1_new_rating - player1_rating)
    ratings.loc[player2, 'change'] = int(player2_new_rating - player2_rating)
    ratings.loc[player3, 'change'] = int(player3_new_rating - player3_rating)
    ratings.loc[player4, 'change'] = int(player4_new_rating - player4_rating)
    
    # update elo_history
    new_data = [{'date': row['Date'], 'player': player, 'elo': elo} 
                for player, elo in zip([player1, player2, player3, player4], 
                                       [player1_new_rating, player2_new_rating, 
                                        player3_new_rating, player4_new_rating])]
    new_df = pd.DataFrame(new_data)
    elo_history = pd.concat([elo_history, new_df], ignore_index=True)
    
    
    games[index] = [player1_new_rating, player2_new_rating, player3_new_rating, player4_new_rating]
    
    #if index > (len(game_log) - 5):
        #display(game_log.iloc[index-5:index+1])
        #print(f'k{[k1, k2, k3, k4]}')
        #display(ratings.sort_values('elo', ascending=False))
        
  
# Update leaderboards
ratings = ratings.sort_values('elo', ascending=False)

In [ ]:
ratings['win_rate'] = round(ratings['wins'] / ratings['total_games'] * 100, 2)
ratings.to_csv('../data/leaderboard - 2023.csv')

# Plots

In [ ]:
# Convert date column to datetime
elo_history['date'] = pd.to_datetime(elo_history['date'])

# floor the time part of datetime, keep only date
elo_history['date'] = elo_history['date'].dt.floor('d')

# Players above and below 1500
players_above_10_games = ratings[(ratings['total_games'] > 10)]
players_top = players_above_10_games.iloc[:int(len(players_above_10_games)/2)]
players_bot = players_above_10_games.iloc[int(len(players_above_10_games)/2):]

player_split = [players_top, players_bot]

for i, df in enumerate(player_split):
    specific_player_df = elo_history[elo_history['player'].isin(df.index.values)]
    
    plt.figure(figsize=(15,8))
    sns.lineplot(data=specific_player_df, x='date', y='elo', hue='player', errorbar=None)

    # Adding title and labels

    plt.title('ELO ranking (players with >10 games)')
    plt.xlabel('Date')
    plt.ylabel('ELO ranking')

    # Rotate date labels on X-axis
    plt.xticks(rotation=45, ha='right')

    # This sets major ticks format for the x axis
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d/%m/%Y'))

    plt.savefig(fname=f"../data/plot_{i}")